# Analyze scRNS-seq jointly with CRISPRa screening result

In [ ]:
import lamindb as ln
import scanpy as sc

ln.track("F4L3oC6QsZvQ", project="Schmidt22")

Load hits from phenotypic screen.

In [ ]:
artifact_hits = ln.Artifact.get(key="schmidt22/gws-crispr-ifng-hits.parquet")
artifact_hits.describe()
screen_hits = artifact_hits.load()

Load the scRNA-seq data.

In [ ]:
artifact_scrna = ln.Artifact.get(
    key="schmidt22_perturbseq/schmidt22_perturbseq_ensembl_gene_ids.h5ad"
)
artifact_scrna.describe()
adata = artifact_scrna.load()

Run an analysis.

In [ ]:
sc.tl.score_genes(
    adata,
    adata.var_names.intersection(
        screen_hits["target_gene_symbol"],
        score_name="enrichment_score_for_screen_results",
    ).tolist(),
)

In [ ]:
adata.obs["cluster_name"].cat.categories.to_list()

In [ ]:
curated_names = [
    name.split(":")[1].strip() for name in adata.obs["cluster_name"].cat.categories
]

In [ ]:
curated_names

In [ ]:
sc.pl.matrixplot(
    adata, groupby="cluster_name", var_names=["enrichment_score_for_screen_results"]
)

In [ ]:
sc.pl.umap(adata, color=["score", "cluster_name"])

Save the output.

In [ ]:
artifact_result = ln.Artifact.from_anndata(
    adata, key="schmidt22/analyzed_data.h5ad"
).save()
artifact_result.describe()

In [ ]:
artifact_result.view_lineage()

In [ ]:
ln.finish()